In [1]:
import ROOT
from fit_utils import make_observed_histogram, hist_to_np, make_asimov_np, make_asimov_fast, make_1D_projections, make_observables, make_new_asimov_observables_np, make_THn_like, pol_idx

OBJ: TStyle	ildStyle	ILD Style : 0 at: 0x8d56b20


In [2]:
run_toy_diagnostics = False
# run_toy_diagnostics = True

In [3]:
# TODO: make it possible to fit multiple runs simultaneously
run = {
    "lumi": 5000,
    # "lumi": 25000,
    "e_pol": 0.,
    "p_pol": 0.,
}
n_bins = 65

# https://arxiv.org/pdf/1506.07830
ilc_250_h20_lumi = 2000
ilc_250_h20 = [
    {
        "lumi": ilc_250_h20_lumi * 0.675,
        "e_pol": -0.8,
        "p_pol": 0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.225,
        "e_pol": 0.8,
        "p_pol": -0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.05,
        "e_pol": -0.8,
        "p_pol": -0.3,
    },
    {
        "lumi": ilc_250_h20_lumi * 0.05,
        "e_pol": 0.8,
        "p_pol": 0.3,
    },
]

# run = ilc_250_h20[0]

In [4]:
parameters = {
    "g1z": 0.0,
    "ka": 0.0,
    "la": 0.0,
}
obs_names = [
    "O_g1z_pos_1em05",
    "O_ka_pos_1em05",
    "O_la_pos_1em05",
]
# input_path = "data/histograms/full/raw_histograms_nb65_range_cut_1d.root"
input_path = "data/histograms/full/raw_histograms.root"
signal_cat = "4f_sw_sl_signal"
signal_processes = [
    "4f_sw_sl_eLpL_signal",
    "4f_sw_sl_eLpR_signal",
    "4f_sw_sl_eRpL_signal",
    "4f_sw_sl_eRpR_signal",
    ]
backgrounds = [
    "4f_sl_bkg",
    "4f_not_sl",
    "2f",
    "3f",
    "5f",
    "6f",
    "higgs",
]

In [5]:
signal_histograms = {}
signal_meta = {}
oo_matrix = {}
template_parameters = {}
background_histograms = {}
with ROOT.TFile(input_path) as input_file:
    # take care of signals first
    signal_dir = input_file[signal_cat]
    for process_name in signal_processes:
        signal_histos = {}
        p_dir = signal_dir[process_name]
        obs = "obs_ND"
        # very consistent ownership model of root requires us to do this
        histo = p_dir[obs]
        # not needed on nD???
        # histo.SetDirectory(ROOT.nullptr)
        signal_histo = histo
        meta_dir = p_dir["meta"]
        obs_meta = {}
        for key in meta_dir.GetListOfKeys():
            key_name = key.GetName()
            obs_meta[key_name] = meta_dir[key_name]
        signal_histograms[process_name] = signal_histo
        signal_meta[process_name] = obs_meta
        # get OO matrix
        mat_dir = input_file["oo_matrix"]
        oo_matrix[process_name] = mat_dir[process_name]
    # get template parameters
    # unfortunately here we have the loops the other way around
    template_dir = input_file["template_parametrisations"]
    for obs in obs_names:
        obs_dir = template_dir[obs]
        p_pars = {}
        for process_name in signal_processes:
            p_dir = obs_dir[process_name]
            pars = {}
            for par in parameters.keys():
            # very consistent ownership model of root requires us to do
                par_hist = p_dir[par]
                par_hist.SetDirectory(ROOT.nullptr)
                pars[par] = par_hist
            p_pars[process_name] = pars
        template_parameters[obs] = p_pars
    for bkg in backgrounds:
        bkg_dir = input_file[bkg]
        bkg_hist = [make_THn_like(list(signal_histograms.values())[0]) for i in range(4)]
        for process in bkg_dir.GetListOfKeys():
            p_dir = bkg_dir[process.GetName()]
            # should contain meta, O_bla... and obs_ND
            meta = p_dir["meta"]
            h = p_dir["obs_ND"]
            # divide h by meta["lumi"]
            h.Scale(1. / meta["lumi"].GetVal())
            # figure out which pol hist to add this to from e_pol and_p_pol
            e_pol = meta["e_pol"].GetVal()
            p_pol = meta["p_pol"].GetVal()
            bkg_hist[pol_idx(e_pol, p_pol)].Add(h)
        background_histograms[bkg] = bkg_hist

print(signal_histograms)
print(signal_meta)
print(oo_matrix)
print(template_parameters)
print(background_histograms)

{'4f_sw_sl_eLpL_signal': <cppyy.gbl.THnT<double> object at 0xad855d0>, '4f_sw_sl_eLpR_signal': <cppyy.gbl.THnT<double> object at 0xad0e4e0>, '4f_sw_sl_eRpL_signal': <cppyy.gbl.THnT<double> object at 0xaf5d160>, '4f_sw_sl_eRpR_signal': <cppyy.gbl.THnT<double> object at 0xad0bc60>}
{'4f_sw_sl_eLpL_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xaee4a80>, 'e_pol': <cppyy.gbl.TParameter<float> object at 0xaedbbf0>, 'p_pol': <cppyy.gbl.TParameter<float> object at 0xae740e0>}, '4f_sw_sl_eLpR_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xaea4440>, 'e_pol': <cppyy.gbl.TParameter<float> object at 0xaf00130>, 'p_pol': <cppyy.gbl.TParameter<float> object at 0xaf001b0>}, '4f_sw_sl_eRpL_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xaeccec0>, 'e_pol': <cppyy.gbl.TParameter<float> object at 0xaeccf80>, 'p_pol': <cppyy.gbl.TParameter<float> object at 0xaecd000>}, '4f_sw_sl_eRpR_signal': {'lumi': <cppyy.gbl.TParameter<float> object at 0xae71490>, 'e_pol': <cppyy.gbl

In [6]:
bkg_1d = [[make_1D_projections(h) for h in bkg] for bkg in background_histograms.values()]
# need to transpose inner 2
bkg_1d = [list(map(list, zip(*foo))) for foo in bkg_1d]
print(bkg_1d)

[[[<cppyy.gbl.TH1D object at 0xb0f9cb0>, <cppyy.gbl.TH1D object at 0xb07d4b0>, <cppyy.gbl.TH1D object at 0xb033e00>, <cppyy.gbl.TH1D object at 0xb0071f0>], [<cppyy.gbl.TH1D object at 0xb130760>, <cppyy.gbl.TH1D object at 0xb01fdf0>, <cppyy.gbl.TH1D object at 0x94c8a20>, <cppyy.gbl.TH1D object at 0xb1014f0>], [<cppyy.gbl.TH1D object at 0xb0b3370>, <cppyy.gbl.TH1D object at 0xafc15b0>, <cppyy.gbl.TH1D object at 0x133a11d0>, <cppyy.gbl.TH1D object at 0xb03f550>]], [[<cppyy.gbl.TH1D object at 0x94bead0>, <cppyy.gbl.TH1D object at 0xb021760>, <cppyy.gbl.TH1D object at 0xaf59190>, <cppyy.gbl.TH1D object at 0xba60e00>], [<cppyy.gbl.TH1D object at 0x5eaa380>, <cppyy.gbl.TH1D object at 0xaf178a0>, <cppyy.gbl.TH1D object at 0x16d8c500>, <cppyy.gbl.TH1D object at 0xb182d10>], [<cppyy.gbl.TH1D object at 0xb916a40>, <cppyy.gbl.TH1D object at 0xba99190>, <cppyy.gbl.TH1D object at 0xb175870>, <cppyy.gbl.TH1D object at 0xb059660>]], [[<cppyy.gbl.TH1D object at 0xba68990>, <cppyy.gbl.TH1D object at 0xb

In [7]:
h_obs = make_observed_histogram(signal_histograms, signal_meta, run)
h_obs_np = hist_to_np(h_obs)
h_asimov_np = make_asimov_np(h_obs_np)
h_asimov = make_asimov_fast(h_obs, 1234)

{ @0x7ffcac8109f0, @0x7ffcac8109f0, @0x7ffcac8109f0 } 3


In [8]:
import numpy as np
h_example = make_1D_projections(h_obs)[0]
bin_centers = np.asarray([h_example.GetBinCenter(i+1) for i in range(h_example.GetNbinsX())])

In [9]:
signal_1d_histograms = [make_1D_projections(h) for h in signal_histograms.values()]
# ugly hack to "transpose" that list[list]
signal_1d_histograms = list(map(list, zip(*signal_1d_histograms)))
print(signal_1d_histograms)
signal_lumi = [par["lumi"].GetVal() for par in signal_meta.values()]
print(signal_lumi)
oo_matrices = list(oo_matrix.values())
print(oo_matrices)
template_param = [[[ph for ph in cpl_h.values()] for cpl_h in hel_h.values()] for hel_h in template_parameters.values()]
print(template_param)

[[<cppyy.gbl.TH1D object at 0x21bf44d0>, <cppyy.gbl.TH1D object at 0x21bbe290>, <cppyy.gbl.TH1D object at 0x21ac17a0>, <cppyy.gbl.TH1D object at 0x21add620>], [<cppyy.gbl.TH1D object at 0x21a36c50>, <cppyy.gbl.TH1D object at 0x21b9ced0>, <cppyy.gbl.TH1D object at 0x21ba4080>, <cppyy.gbl.TH1D object at 0x21c03e40>], [<cppyy.gbl.TH1D object at 0x21adbbe0>, <cppyy.gbl.TH1D object at 0x21bb3350>, <cppyy.gbl.TH1D object at 0x21bb1f60>, <cppyy.gbl.TH1D object at 0x21ac3240>]]
[598.3264770507812, 100.83772277832031, 4009.4052734375, 696.610107421875]
[<cppyy.gbl.ROOT.Math.SVector<double,6> object at 0x8d223b0>, <cppyy.gbl.ROOT.Math.SVector<double,6> object at 0xae77260>, <cppyy.gbl.ROOT.Math.SVector<double,6> object at 0xaf1d8b0>, <cppyy.gbl.ROOT.Math.SVector<double,6> object at 0xaf06f60>]
[[[<cppyy.gbl.TH1D object at 0xa10c4c0>, <cppyy.gbl.TH1D object at 0xafae200>, <cppyy.gbl.TH1D object at 0xa08d8c0>], [<cppyy.gbl.TH1D object at 0x9f47fe0>, <cppyy.gbl.TH1D object at 0x987bb70>, <cppyy.gbl

In [10]:
ROOT.gInterpreter.Declare("#include \"fit.h\"")

True

In [11]:
fun2 = ROOT.fit_fun2[f"3, {n_bins}, 3"](signal_1d_histograms, signal_lumi, oo_matrices, template_param, bkg_1d)

start constructor
finished constructor


In [12]:
if run_toy_diagnostics:
    n_obs = 3
    n_toys = 10
    # h_chi2 = ROOT.TH1D("", ";#chi^{2}", 100, -10., 10.)
    h_chi2 = ROOT.TH1D("", ";#chi^{2}", 50, 0., 10.)
    h_prob = ROOT.TH1D("", ";probability", 50, 0., 1.)
    h_diff0 = ROOT.TH1D("", ";obs_asimov - obs [0]", 100, -10000., 10000.)
    h_diff1 = ROOT.TH1D("", ";obs_asimov - obs [1]", 100, -10000., 10000.)
    h_diff2 = ROOT.TH1D("", ";obs_asimov - obs [2]", 100, -10000., 10000.)
    obs_initial = make_observables(make_1D_projections(h_obs))
    C = np.zeros((n_obs, n_obs))
    v = np.zeros(n_obs)
    for seed in range(n_toys):
        # obs = make_new_asimov_observables(h_obs, seed)
        obs = make_new_asimov_observables_np(h_obs_np, bin_centers)
        pars = list(run.values()) + [0., 0., 0.] + [1.]
        chi2 = fun2(obs, pars)
        # print(obs, chi2)
        prob = ROOT.Math.chisquared_cdf(chi2, 3)
        h_chi2.Fill(chi2)
        h_prob.Fill(prob)
        diff = [0.] * n_obs
        for i in range(n_obs):
            diff[i] = obs[i] - obs_initial[i]
            v[i] += diff[i]
        for i in range(n_obs):
            for j in range(n_obs):
                C[i,j] += diff[i] * diff[j]
        h_diff0.Fill(diff[0])
        h_diff1.Fill(diff[1])
        h_diff2.Fill(diff[2])
    C /= n_toys
    v /= n_toys

In [13]:
if run_toy_diagnostics:
    c_chi2 = ROOT.TCanvas()
    f = ROOT.TF1("chi2", "[0] * ROOT::Math::chisquared_pdf(x, [1])", 0., 10.)
    f.SetParameters(10000/5., 3.)
    # f.FixParameter(1, 3.)
    h_chi2.Fit(f)
    h_chi2.Draw()
    # f.Draw("same")
    c_chi2.Draw()
    # c_chi2.SaveAs("plots/fit/diagnostics/chi2.pdf")

    c_prob = ROOT.TCanvas()
    h_prob.SetMinimum(0)
    h_prob.Draw()
    c_prob.Draw()
    # c_prob.SaveAs("plots/fit/diagnostics/prob.pdf")

    c_diff0 = ROOT.TCanvas()
    h_diff0.Fit("gaus")
    h_diff0.Draw()
    c_diff0.Draw()
    # c_diff0.SaveAs("plots/fit/diagnostics/diff0.pdf")

    c_diff1 = ROOT.TCanvas()
    h_diff1.Fit("gaus")
    h_diff1.Draw()
    c_diff1.Draw()
    # c_diff1.SaveAs("plots/fit/diagnostics/diff1.pdf")

    c_diff2 = ROOT.TCanvas()
    h_diff2.Fit("gaus")
    h_diff2.Draw()
    c_diff2.Draw()
    # c_diff2.SaveAs("plots/fit/diagnostics/diff2.pdf")

In [14]:
# build all the RooFit stuff

# use the initial assumed as true values to init
obs_initial = make_observables(make_1D_projections(h_obs))
# define observable parameters
obs_pars = []
for i, o in enumerate(obs_initial):
    name = obs_names[i]
    # lets just choose #pm 10% where ever we're not sure
    min_o = o*0.9
    max_o = o*1.1
    if o < 0:
        # swap around to avoid RooFit printing an error when doing it
        min_o, max_o = max_o, min_o
    ob = ROOT.RooRealVar(name, name, min_o, max_o)
    obs_pars.append(ob)

# define run parameters
lumi_par = ROOT.RooRealVar("lumi", "lumi", run["lumi"], 0.9*run["lumi"], 1.1 * run["lumi"])
lumi_par.setConstant()
e_pol_par = ROOT.RooRealVar("e_pol", "e_pol", run["e_pol"], -1., 1.)
e_pol_par.setConstant()
p_pol_par = ROOT.RooRealVar("p_pol", "p_pol", run["p_pol"], -1., 1.)
p_pol_par.setConstant()

# define coupling parameters
coupling_pars = []
for name, value in parameters.items():
    # that should be tight enough...
    cpl = ROOT.RooRealVar(name, name, value, -0.5, 0.5)
    coupling_pars.append(cpl)

# define nuisance parameters
nuisance_pars = []
# signal only so far
nu_par = ROOT.RooRealVar("mu_signal", "mu_signal", 1., 0.9, 1.1)
nu_par.setConstant()
nuisance_pars.append(nu_par)

all_pars = [lumi_par, e_pol_par, p_pol_par] + coupling_pars + nuisance_pars
obs_and_pars = obs_pars + all_pars
obs_and_pars_arglist = ROOT.RooArgList(obs_and_pars)

In [15]:
fun2_functor = ROOT.Math.Functor(fun2, len(obs_and_pars))
chi2_fun = ROOT.RooFit.bindFunction("chi2_fun", fun2_functor, obs_and_pars_arglist)

In [16]:
# should be constant with the number of observables
chi2_ndf = ROOT.RooRealVar("chi2_ndf", "chi2_ndf", len(obs_pars))
model = ROOT.RooChiSquarePdf("chi2_pdf", "chi2_pdf", chi2_fun, chi2_ndf)

In [17]:
def make_ttree_from_obs(obs):
    from array import array
    tree = ROOT.TTree("tree", "tree")
    branch_pointers = []
    for i, o in enumerate(obs_initial):
        name = obs_names[i]
        p = array("d", [0])
        tree.Branch(name, p, f"{name}/D")
        p[0] = o
        branch_pointers.append(p)
    tree.Fill()
    return tree

In [18]:
obs = make_new_asimov_observables_np(h_obs_np, bin_centers)
obs_tree = make_ttree_from_obs(obs)
ds = ROOT.RooDataSet("ds", "ds", obs_pars, Import=obs_tree)

In [19]:
# model.fitTo(ds)
model.getVal()
model.Print("t")

ERROR incorrect number of background norm factors given: 1 (needed: 7)
0x24a2e240 RooChiSquarePdf::chi2_pdf = 5.11417e-12 [Auto,Dirty] 
  0x24585770/V- RooFunctorBinding::chi2_fun = 1.64335e-22 [Auto,Clean] 
    0x2348fed0/V- RooRealVar::O_g1z_pos_1em05 = -328313
    0x23178cc0/V- RooRealVar::O_ka_pos_1em05 = -816884
    0x23016740/V- RooRealVar::O_la_pos_1em05 = -404445
    0x237b2f90/V- RooRealVar::lumi = 5000
    0x233d80f0/V- RooRealVar::e_pol = 0
    0x22c6c8d0/V- RooRealVar::p_pol = 0
    0x21bfc780/V- RooRealVar::g1z = 0
    0x23754e90/V- RooRealVar::ka = 0
    0x2374ac80/V- RooRealVar::la = 0
    0x23748bf0/V- RooRealVar::mu_signal = 1
  0x244b5bf0/V- RooRealVar::chi2_ndf = 3


In [20]:
# model.fitTo(ds)

In [21]:
# abuse ConditionalObservables to inhibit integration
nll = model.createNLL(ds, EvalBackend="cpu", ConditionalObservables=obs_pars)
# nll = model.createNLL(ds, EvalBackend="legacy")
# nll = model.createNLL(ds, EvalBackend="codegen")
# nll = model.createNLL(ds, EvalBackend="codegen_no_grad")
# not nll but chi2
# nll = chi2_fun

[#1] INFO:Fitting -- RooAbsPdf::fitTo(chi2_pdf) fixing normalization set for coefficient determination to observables in data
[#1] INFO:Fitting -- using generic CPU library compiled with no vectorizations
[#1] INFO:Fitting -- Creation of NLL object took 3.31202 ms


In [22]:
nll_minimizer = ROOT.RooMinimizer(nll)

[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_chi2_pdf_ds) Summation contains a RooNLLVar, using its error level


In [23]:
%time
nll_minimizer.migrad()

CPU times: user 2 μs, sys: 0 ns, total: 2 μs
Wall time: 3.34 μs


1

Minuit2Minimizer: Minimize with max-calls 1500 convergence for edm < 1 strategy 1
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed

Info in <Minuit2>: MnSeedGenerator Computing seed using NumericalGradient calculator
Info in <Minuit2>: MnSeedGenerator Evaluated function and gradient in 276.499 μs
Info in <Minuit2>: MnSeedGenerator Initial state: FCN =       26.00083449 Edm =   1.450850962e-13 NCalls =     13
Info in <Minuit2>: NegativeG2LineSearch Doing a NegativeG2LineSearch since one of the G2 component is negative
Info in <Minuit2>: NegativeG2LineSearch Done after 143.478 μs
Info in <Minuit2>: MnSeedGenerator Negative G2 found - new state: 
  Minimum value : 1.419048725
  Edm           : 0.0003329777234
  Internal parameters:	[  -0.001006789604 -0.0006427279946                0]	
  Internal gradient  :	[      17.16441877      5.549430442     -12.10956769]	
  Internal covariance matrix:
[[  1.4920338e-06              0              0]
 [              0  1.4545581e-05              0]
 [              0              0  3.0304058e-06]]]
Info in <Minuit2>: MnSeedGenerator Initial state  
  Minimum value : 1.419048725


In [24]:
%time
# nll_minimizer.hesse()
# nll_minimizer.minos(coupling_pars)

CPU times: user 1 μs, sys: 1 μs, total: 2 μs
Wall time: 2.86 μs


In [25]:
# nll.Print("t")

In [26]:
pll0 = nll.createProfile({coupling_pars[0]})
pll1 = nll.createProfile({coupling_pars[1]})
pll2 = nll.createProfile({coupling_pars[2]})

In [27]:
frame0 = coupling_pars[0].frame(Range=(-0.004, 0.004))
frame1 = coupling_pars[1].frame(Range=(-0.004, 0.004))
frame2 = coupling_pars[2].frame(Range=(-0.004, 0.004))
nll.plotOn(frame0, ShiftToZero=True)
nll.plotOn(frame1, ShiftToZero=True)
nll.plotOn(frame2, ShiftToZero=True)

ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR 

In [28]:
pll0.plotOn(frame0, LineColor="r")
pll1.plotOn(frame1, LineColor="r")
pll2.plotOn(frame2, LineColor="r")

[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) Creating instance of MINUIT
[#1] INFO:Fitting -- RooAddition::defaultErrorLevel(nll_chi2_pdf_ds) Summation contains a RooNLLVar, using its error level
[#1] INFO:Minimization -- RooProfileLL::evaluate(RooEvaluatorWrapper_Profile[g1z]) determining minimum likelihood for current configurations w.r.t all observable
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background norm factors given: 1 (needed: 7)
ERROR incorrect number of background

In [29]:
c0 = ROOT.TCanvas()
frame0.SetMinimum(0)
frame0.SetMaximum(4)
frame0.Draw()
c0.Draw()
# c0.SaveAs("plots/fit/ll_pll.pdf(")

c1 = ROOT.TCanvas()
frame1.SetMinimum(0)
frame1.SetMaximum(4)
frame1.Draw()
c1.Draw()
# c1.SaveAs("plots/fit/ll_pll.pdf")

c2 = ROOT.TCanvas()
frame2.SetMinimum(0)
frame2.SetMaximum(4)
frame2.Draw()
c2.Draw()
# c2.SaveAs("plots/fit/ll_pll.pdf)")